This map has been hand edited in QGIS to remove extraneous features and to greatly simplify the polygons.

Convert the shapefile to geojson

In [1]:
# npm install -g shapefile

!shp2json '../raw_data/map/simplified/simplified.shp' -o '../raw_data/map/simplified.json'

Split up the geojson file so that each line ends with a `\n` character

In [2]:
# npm install -g ndjson-cli

!ndjson-split 'd.features' \
    < '../raw_data/map/simplified.json' \
    > '../raw_data/map/simplified.ndjson'

In [3]:
!wc -l ../raw_data/map/simplified.ndjson

     177 ../raw_data/map/simplified.ndjson


In [4]:
!rm ../raw_data/map/simplified.json

Add a Code key for each country, so that it can be used for the join and reduce the amount of properties for each country, to just the name `NAME_EN`, country code `ADM0_A3`, - this has been edited in qgis - and region `REGION_UN`.

In [5]:
!ndjson-map 'd.code = d.properties.ADM0_A3, d' < ../raw_data/map/simplified.ndjson \
    | ndjson-map 'd.properties = { name:d.properties.NAME_EN, code:d.properties.ADM0_A3, altcode:d.properties.GU_A3, region:d.properties.REGION_UN }, d' \
    > ../raw_data/map/temp.ndjson



In [6]:
!rm ../raw_data/map/simplified.ndjson

In [7]:
!mv ../raw_data/map/temp.ndjson ../raw_data/map/simplified.ndjson

Convert back from ndjson

In [8]:
!ndjson-reduce < ../raw_data/map/simplified.ndjson \
    | ndjson-map '{type: "FeatureCollection", features: d}' \
    > ../raw_data/map/simplified.json

In [9]:
!rm ../raw_data/map/simplified.ndjson

Convert from from geojson to topojson

In [10]:
# npm install -g topojson

!geo2topo ../raw_data/map/simplified.json > ../raw_data/map/simplified-topo.json

Move into the data file

In [11]:
!mv ../raw_data/map/simplified-topo.json ../src/data/simplified-topo.json